This file has 2 parts, run the one you want.

VOSA requires following format for uploading the photometric information files
 ----------------------------------------------------------------------------
| object  | RA  | DEC | dis | Av | filter | flux | error | pntopts | objopts |
| ---     | --- | --- | --- | ---| ---    | ---  | ---   | ---     | ---     |
| ---     | --- | --- | --- | ---| ---    | ---  | ---   | ---     | ---     |

Identify the filters in the files you are uploading (create the "filter_list" accordingly)

Create a file with "name, ra, dec, magnitudes and magnitude errors".
This "photomety_file" be converted to VOSA format
    Note: This code is for a cluster, hence distance and extinction is kept constant
    
The VOSA_input.txt file has magnitudes in the "flux" column. So while uploading to VOSA, keep "file type" as "magnitudes"

In [2]:
# ---------------------------------------------------------------------- #
# -------------------- Creating VOSA upload files ---------------------- #
# ---------------------------------------------------------------------- #
import pandas as pd

DISTANCE = '831.76+-11'  # in pc
A_V = 0.1736    # = 3.1*E(B-V)
e_A_V = 0

photomety_file = 'data/example_photomety_file.csv'    # Has name, ra, dec, magnitude, errors
photometry = pd.read_csv(photomety_file, engine='python')
print (list(photometry.columns.values))

['name', 'ra', 'dec', 'B', 'e_B', 'V', 'e_V', 'I', 'e_I', 'U', 'e_U', 'R', 'e_R', 'GALEX_NUV', 'e_GALEX_NUV', 'GALEX_FUV', 'e_GALEX_FUV', 'F148W', 'e_F148W', 'F154W', 'e_F154W', 'F169M', 'e_F169M']


In [19]:
# Make the three lists. 1) VOSA names for filters, 2)names of the magnitude colums, 3)names of the error column
filter_list = ['KPNO/Mosaic.B', 'KPNO/Mosaic.V','KPNO/Mosaic.I', 
               'KPNO/Mosaic.U','KPNO/Mosaic.R','GALEX/GALEX.FUV',
               'GALEX/GALEX.NUV','Astrosat/UVIT.F148W','Astrosat/UVIT.F154W',
               'Astrosat/UVIT.F169M']
mag_list = ['B', 'V', 'I',
            'U','R','GALEX_NUV',
            'GALEX_FUV','F148W','F154W',
            'F169M']
err_list = ['e_B', 'e_V', 'e_I',
            'e_U','e_R','e_GALEX_NUV',
            'e_GALEX_FUV','e_F148W','e_F154W',
            'e_F169M']

flux_type = 'mag'     # 'mag' or 'jy' or 'erg'
'''
mag --> the magnitudes of each filters assuming vega/AB type as same as http://svo2.cab.inta-csic.es/theory/fps/pavosa.php
jy  --> flux density in jy. This requires use of zero_point_list approporiate for your filter
erg --> flux density in erg/cm^2/s/A.
'''
# Update following values to your filters if you are not planning to upload in magnitudes 
zero_point_list_jy = [3954.5,3632,2384.1,
                      1681.2,2945.8,3631,
                      3631,3631,3631,
                      3631]  # in jy
zero_point_list_erg = [6.24076e-9,3.78104e-9,1.08913e-9,
                       3.82411e-9,2.11969e-9,2.04929e-8,
                       4.53663e-8,4.60576e-8,4.42378e-8,
                       4.21999e-8]  # in erg/cm2/s/A
wavelength = [4358.52,5366.30,8100.90,
              3630.40,6454.66,2304.74,
              1549.02,1481.00,1541.00,
              1606.08] # effective wavelength in A

'\n\n'

In [20]:
# combining data from all stars to make the VOSA upload file 
VOSA = pd.DataFrame(columns = ['object', 'RA', 'DEC','dis','Av','filter','flux','error','pntopts','objopts']) 
for i in range (0,len(photometry)):
    for j in range (0,len(filter_list)):
        if flux_type=='mag':
            flux = photometry[mag_list[j]][i]
            error = photometry[err_list[j]][i]
        if flux_type =='jy': # please note that the magnitude errors are asymmetric in nature when converted to flux
            flux = zero_point_list_jy[j]* 10**(-0.4*photometry[mag_list[j]][i])
            error = zero_point_list_jy[j]* 10**(-0.4*(photometry[mag_list[j]][i]-photometry[err_list[j]][i])) - flux
        if flux_type == 'erg': # please note that the magnitude errors are asymmetric in nature when converted to flux
            flux = zero_point_list_erg[j]* 10**(-0.4*photometry[mag_list[j]][i])
            error = zero_point_list_erg[j]* 10**(-0.4*(photometry[mag_list[j]][i]-photometry[err_list[j]][i])) - flux            
            
        VOSA = VOSA.append({'object': photometry['name'][i], 
                            'RA':photometry['ra'][i], 
                            'DEC':photometry['dec'][i],
                            'dis':DISTANCE,
                            'Av':str(A_V)+'+-'+str(e_A_V),
                            'filter':filter_list[j],
                            'flux':flux,
                            'error':error,
                            'pntopts':'---',
                            'objopts': '---'},ignore_index = True)
VOSA.fillna('---', inplace=True)

print (VOSA)
VOSA.to_csv('data/example_VOSA_input_file_'+flux_type+'.txt', header=None, index=None, sep=' ')
'''
Now upload the file at http://svo2.cab.inta-csic.es/theory/vosa/index.php?action=myfiles&otype=star&seeall= 
Make sure to change the File type: To magnitude or Flux (jy) or Flux (erg) accordingly 
Keep SED Type: Flux vs Lambda 
Select the file and search through VO for all possible detections Look at the SEDs, 
Possibly remove some telescopes (SDSS creates problems most of the times)
'''

     object        RA      DEC         dis         Av               filter  \
0  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0        KPNO/Mosaic.B   
1  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0        KPNO/Mosaic.V   
2  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0        KPNO/Mosaic.I   
3  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0        KPNO/Mosaic.U   
4  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0        KPNO/Mosaic.R   
5  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0      GALEX/GALEX.FUV   
6  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0      GALEX/GALEX.NUV   
7  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0  Astrosat/UVIT.F148W   
8  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0  Astrosat/UVIT.F154W   
9  WOCS2002  132.8492  11.8304  831.76+-11  0.1736+-0  Astrosat/UVIT.F169M   

     flux  error pntopts objopts  
0    12.3    ---     ---     ---  
1   11.54    ---     ---     ---  
2   11.06    ---     ---     ---  
3

'\nNow upload the file at http://svo2.cab.inta-csic.es/theory/vosa/index.php?action=myfiles&otype=star&seeall= \nMake sure to change the File type: To magnitude or Flux (jy) or Flux (erg) accordingly \nKeep SED Type: Flux vs Lambda \nSelect the file and search through VO for all possible detections Look at the SEDs, \nPossibly remove some telescopes (SDSS creates problems most of the times)\n'

# Creating Synthetic Photometry model files
- Download synthetic photometry for all filters from http://svo2.cab.inta-csic.es/theory/newov2/syph.php  
- unzip the files in "models" folder
- Use following code to combine the individual files (of each temperature) into a single file for particular model parameters.

In [70]:
import os
import pandas as pd

def create_synthetic_photometry(model,logg,Z,save=True,verbose=False):
    if model=='Koe':     # List of all available temperatures for Koester model
        temp_list=['05000','05250','05500','05750','06000','06250','06500','06750','07000','07250','07500','07750','08000','08250','08500','08750','09000','09250','09500','09750','10000','10250','10500','10750','11000','11250','11500','11750','12000','12250','12500','12750','13000','13250','13500','13750','14000','14250','14500','14750','15000','15250','15500','15750','16000','16250','16500','16750','17000','17250','17500','17750','18000','18250','18500','18750','19000','19250','19500','19750','20000','21000','22000','23000','24000','25000','26000','27000','28000','29000','30000','32000','34000','35000','36000','38000','40000','45000','50000','60000','70000','80000']
    if model == 'Kr':    # List of all available temperatures for Kurucz model
        temp_list=['3500','3750','4000','4250','4500','4750','5000','5250','5500','5750','6000','6250','6500','6750','7000','7250','7500','7750','8000','8250','8500','8750','9000','9250','9500','9750','10000','10250','10500','10750','11000','11250','11500','11750','12000','12250','12500','12750','13000','14000','15000','16000','17000','18000','19000','20000','21000','22000','23000','24000','25000','26000','27000','28000','29000','30000','31000','32000','33000','34000','35000','36000','37000','38000','39000','40000','41000','42000','43000','44000','45000','46000','47000','48000','49000','50000']
        
    flux_model =  pd.DataFrame()

    for i in range (0,len(temp_list)):
        if model=='Koe':
            _model_files = 'C:/Users/Zert W/Desktop/python/Synthetic_photometry_models/koester2_phot_1605778413.1972/koester2_da'+str(temp_list[i])+'_'+logg+'.dk.phot.dat'   # Check the names for logg, Z, model etc. 
        if model == 'Kr':
            _model_files= 'C:/Users/Zert W/Desktop/python/Synthetic_photometry_models/Kurucz_phot_1605787543.3471/Kurucz_f'+Z+'k2odfnew.pck.teff='+str(temp_list[i])+'..logg='+logg+'0000.phot.dat'
        if not os.path.isfile(_model_files):
            #print (temp_list[i]),
            pass
        else:
            _data_flux = pd.read_csv(_model_files,comment='#',engine='python', header=None, delim_whitespace= True,skipinitialspace=True)
            _data_flux.columns = ['filter', 'wave', 'flux']
            _data_flux = _data_flux.set_index('filter')
            if i==0:
                flux_model['filter'] = _data_flux.index
                flux_model = flux_model.set_index('filter')
                flux_model['wave'] = _data_flux['wave']
            flux_model[temp_list[i]]=_data_flux['flux']    # Saving the flux from above file in a new dataframe as a single column
            
    flux_model = flux_model.sort_values(by=['wave'])
    if model == 'Koe':
        output_name = 'models/'+model+'_logg'+str(int(logg)/100.0)+'.csv'
    if model =='Kr':
        output_name = 'models/'+model+'_logg'+logg+'_Z'+Z+'.csv'
    if save:
        flux_model.to_csv(output_name, index=True)
    
    print (output_name)
    return flux_model

## Creating Kurucz model

In [68]:
# For Kr models:
#          logg is like: 3.0, 4.5, 5.0 etc.
#          Z is m05 (-0.5), p00 (0), p05(0.5) etc.
Kr_logg_list=['0.0','0.5','1.0','1.5','2.0','2.5','3.0','3.5','4.0','4.5','5.0']
Kr_Z_list = ['m25','m20','m15','m10','m05','p00','p05']

for logg in Kr_logg_list:
    for Z in Kr_Z_list:
        print (logg,Z)
        create_synthetic_photometry('Kr',logg,Z)


0.0 m25
models/Kr_logg0.0_Zm25.csv
0.0 m20
models/Kr_logg0.0_Zm20.csv
0.0 m15
models/Kr_logg0.0_Zm15.csv
0.0 m10
models/Kr_logg0.0_Zm10.csv
0.0 m05
models/Kr_logg0.0_Zm05.csv
0.0 p00
models/Kr_logg0.0_Zp00.csv
0.0 p05
models/Kr_logg0.0_Zp05.csv
0.5 m25
models/Kr_logg0.5_Zm25.csv
0.5 m20
models/Kr_logg0.5_Zm20.csv
0.5 m15
models/Kr_logg0.5_Zm15.csv
0.5 m10
models/Kr_logg0.5_Zm10.csv
0.5 m05
models/Kr_logg0.5_Zm05.csv
0.5 p00
models/Kr_logg0.5_Zp00.csv
0.5 p05
models/Kr_logg0.5_Zp05.csv
1.0 m25
models/Kr_logg1.0_Zm25.csv
1.0 m20
models/Kr_logg1.0_Zm20.csv
1.0 m15
models/Kr_logg1.0_Zm15.csv
1.0 m10
models/Kr_logg1.0_Zm10.csv
1.0 m05
models/Kr_logg1.0_Zm05.csv
1.0 p00
models/Kr_logg1.0_Zp00.csv
1.0 p05
models/Kr_logg1.0_Zp05.csv
1.5 m25
models/Kr_logg1.5_Zm25.csv
1.5 m20
models/Kr_logg1.5_Zm20.csv
1.5 m15
models/Kr_logg1.5_Zm15.csv
1.5 m10
models/Kr_logg1.5_Zm10.csv
1.5 m05
models/Kr_logg1.5_Zm05.csv
1.5 p00
models/Kr_logg1.5_Zp00.csv
1.5 p05
models/Kr_logg1.5_Zp05.csv
2.0 m25
models/Kr_lo

## Creating Koester model

In [78]:
# For Koe models:
#          logg is like: 700, 750, 850 etc. (meaning 7,8,9 etc.)
#          Z does not matter 
Koe_logg_list = ['650','675','700','725','750','775','800','825','850','875','900','925','950']

for logg in Koe_logg_list:
    print (logg)
    Z='--'
    create_synthetic_photometry('Koe',logg,Z)

650
models/Koe_logg6.5.csv
675
models/Koe_logg6.75.csv
700
models/Koe_logg7.0.csv
725
models/Koe_logg7.25.csv
750
models/Koe_logg7.5.csv
775
models/Koe_logg7.75.csv
800
models/Koe_logg8.0.csv
825
models/Koe_logg8.25.csv
850
models/Koe_logg8.5.csv
875
models/Koe_logg8.75.csv
900
models/Koe_logg9.0.csv
925
models/Koe_logg9.25.csv
950
models/Koe_logg9.5.csv
